In [1]:
import cv2
import os
from matplotlib import pyplot as plt
from predict import *
from getX import getX
from getY import getY
from resize import resize
from PIL import Image, ImageDraw

# data_path = "/media/queahren/DATA/服务外包大赛数据更新/训练数据/img_train_折线"
data_path = "/media/queahren/DATA/DATA_A28_same/images"
# data_path = "/media/queahren/DATA/testData/测试数据/images"
# data_path = "/media/queahren/DATA/MYDATA/images"

# img = cv2.imread("./220/draw.png")
# gray = cv2.imread("./220/draw.png", cv2.IMREAD_GRAYSCALE)

def predImage(img_path):
    assert os.path.exists(img_path), "file: '{}' dose not exist.".format(img_path)
    img = Image.open(img_path)
    return predict(img)


# 不算最后一个数据
def calP(y_nums):
    
    y_nums.sort(key=lambda tup: tup[0], reverse=True)
    if len(y_nums) > 2:
        dy = y_nums[0][0] - y_nums[len(y_nums)-2][0]
        dy_p = y_nums[0][1] - y_nums[len(y_nums)-2][1]
        p = dy/dy_p
    elif len(y_nums) == 2:
        dy = y_nums[0][0] - y_nums[1][0]
        dy_p = y_nums[0][1] - y_nums[1][1]
        p = dy/dy_p
    else:
        print("error")
    return p, y_nums[0]


In [ ]:
filename = os.listdir(data_path)
base_dir = data_path + "/"
font=cv2.FONT_HERSHEY_SIMPLEX

for imgname in filename:
    if(imgname.endswith('.png')):

        print(imgname)

        img = cv2.imread(base_dir + imgname)
        gray = cv2.imread(base_dir + imgname, cv2.IMREAD_GRAYSCALE)

        mean, std = cv2.meanStdDev(gray)
        if mean < 50:
            gray = 255 - gray

        bw = cv2.adaptiveThreshold(
            gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 25, 25)
        img2, ctrs, hier = cv2.findContours(
            bw.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        rects = [cv2.boundingRect(ctr) for ctr in ctrs]

        save_path = "/media/queahren/DATA/DATA_A28_same/nums/" + imgname.split('.')[0] + '/'
        save_path_data = "/media/queahren/DATA/DATA_A28_same/Y_Data/"
        save_path_preview = "/media/queahren/DATA/DATA_A28_same/preview/"
#         save_path = "/media/queahren/DATA/testData/测试数据/nums/" + imgname.split('.')[0] + '/'
#         save_path_data = "/media/queahren/DATA/testData/测试数据/Y_Data/"
#         save_path_preview = "/media/queahren/DATA/testData/测试数据/preview/"
#         save_path = "/media/queahren/DATA/MYDATA/nums/" + imgname.split('.')[0] + '/'
#         save_path_data = "/media/queahren/DATA/MYDATA/Y_Data/"
#         save_path_preview = "/media/queahren/DATA/MYDATA/preview/"
        
        if (os.path.exists(save_path) == False):
            os.mkdir(save_path)
        if (os.path.exists(save_path_data) == False):
            os.mkdir(save_path_data)    

        file0 = open(save_path_data + imgname.split('.')[0] + ".txt",'w+') 
        new_rects = []
        i = 0

        for rect in rects:

            x, y, w, h = rect
            roi = gray[y:y + h, x:x + w]
    #         print(w,h)     
            if (w > 4 and h < 20 and h > 5):
    #             cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 255), 1, 1, 0);
                if w/h >= 1 and w < 22:
                    res = cv2.resize(roi, (2*w, 2*h))
                    res1 = resize(res[0:2*h, 0:w])
                    res2 = resize(res[0:2*h, w:2*w])

                    cv2.imwrite(save_path + '%d.jpg' % i, (res1)) 
                    num, prob = predImage(save_path + '%d.jpg' % i)
                    new_rects.append((x, y, round(w/2), h, num, i))
                    cv2.rectangle(img, (x, y), (x+round(w/2), y+h), (0, 0, 255), 1, 1, 0);
                    cv2.putText(img, '{:.0f}'.format(num), (x, y), cv2.FONT_HERSHEY_DUPLEX, h/25.0, (255, 0, 0))
                    i += 1

                    cv2.imwrite(save_path + '%d.jpg' % i, (res2))
                    num, prob = predImage(save_path + '%d.jpg' % i)
                    new_rects.append((x+round(w/2), y, round(w/2), h, num, i))
                    cv2.rectangle(img, (x+round(w/2), y), (x+w, y+h), (0, 0, 255), 1, 1, 0);
                    cv2.putText(img, '{:.0f}'.format(num), (x+round(w/2), y), cv2.FONT_HERSHEY_DUPLEX, h/25.0, (255, 0, 0))
                    i += 1

                elif w < 15:               
                    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 255), 1, 1, 0);
                    res = resize(roi)
                    cv2.imwrite(save_path + '%d.jpg' % i, res)
                    num, prob = predImage(save_path + '%d.jpg' % i)
                    rect += tuple((num, i))
                    new_rects.append(rect)    
                    cv2.putText(img, '{:.0f}'.format(num), (x, y), cv2.FONT_HERSHEY_DUPLEX, h/25.0, (255, 0, 0))
                    i += 1

        cv2.imwrite(save_path_preview + imgname, img)    
        y_nums = getY(new_rects)
        p, y_num_max = calP(y_nums)
        y_nums, x_p = getX(new_rects)
        print(y_nums, file=file0)
        print(p, file=file0)
        print(y_num_max[1], file=file0)
        print(y_num_max[0], file=file0)
        print(x_p, file=file0)
        file0.close()
    

curve_1.png
curve_10.png
curve_100.png
curve_1001.png
curve_1003.png
curve_1004.png
curve_1005.png
curve_1009.png
curve_101.png
curve_1011.png
curve_1013.png
curve_1014.png
curve_1015.png
curve_1016.png
curve_1018.png
curve_102.png
curve_1020.png
curve_1021.png
curve_1022.png
curve_1023.png
curve_1024.png
line_2734.png
line_2736.png
line_2737.png
line_2739.png
line_274.png
line_2742.png
line_2743.png
line_2744.png
line_2745.png
line_2746.png
line_2747.png
line_2748.png
line_2750.png
line_2751.png
line_2752.png
line_2753.png
line_2756.png
line_2757.png
line_2760.png
line_2761.png
line_2762.png
line_2763.png
line_1979.png
line_198.png
line_1980.png
line_1982.png
line_1983.png
line_1984.png
line_1985.png
line_1986.png
line_1988.png
line_199.png
line_1990.png
line_1991.png
line_1992.png
line_1994.png
line_1995.png
line_1997.png
line_1998.png
line_2.png
line_2001.png
line_2003.png
line_2005.png
line_2007.png
line_2011.png
line_2012.png
line_2015.png
line_2016.png
line_438.png
line_44.png
li

line_2609.png
line_261.png
line_661.png
line_662.png
line_664.png
line_665.png
line_666.png
line_667.png
line_668.png
line_669.png
line_67.png
line_670.png
line_671.png
line_673.png
line_674.png
line_675.png
line_676.png
line_678.png
line_679.png
line_680.png
line_681.png
line_685.png
line_686.png
line_687.png
line_688.png
line_689.png
line_69.png
line_690.png
line_2209.png
line_221.png
line_2210.png
line_2211.png
line_2212.png
line_2213.png
line_2214.png
line_2215.png
line_2216.png
line_2218.png
line_222.png
line_2220.png
line_2221.png
line_2223.png
line_2224.png
line_2225.png
line_2226.png
line_2227.png
line_223.png
line_2765.png
line_2768.png
line_277.png
line_2770.png
line_2772.png
line_2773.png
line_2774.png
line_2775.png
line_2776.png
line_2777.png
line_2778.png
line_2779.png
line_278.png
line_2780.png
line_2781.png
line_2782.png
line_2784.png
line_2785.png
line_398.png
line_399.png
line_40.png
line_400.png
line_401.png
line_403.png
line_404.png
line_405.png
line_406.png
line_407

line_2883.png
line_2884.png
line_2886.png
line_2887.png
line_2888.png
line_289.png
line_2056.png
line_2057.png
line_2058.png
line_206.png
line_2062.png
line_2064.png
line_2065.png
line_2066.png
line_2067.png
line_2069.png
line_2072.png
line_2073.png
line_2074.png
line_2076.png
line_2077.png
line_2078.png
line_2079.png
line_208.png
line_2080.png
line_2081.png
line_1525.png
line_1527.png
line_1528.png
line_153.png
line_1530.png
line_1531.png
line_1532.png
line_1535.png
line_1536.png
line_1537.png
line_1538.png
line_1539.png
line_154.png
line_1540.png
line_1542.png
line_1543.png
line_1544.png
line_1546.png
line_1549.png
line_155.png
line_1551.png
line_1552.png
line_1553.png
line_1555.png
line_1556.png
line_1557.png
line_1558.png
line_156.png
line_1561.png
line_1564.png
line_1565.png
line_1566.png
line_1567.png
line_1887.png
line_1889.png
line_189.png
line_1890.png
line_1891.png
line_1892.png
line_1893.png
line_1894.png
line_1895.png
line_1897.png
line_1898.png
line_1899.png
line_190.png
l

line_2244.png
line_2245.png
line_2246.png
line_2247.png
line_2248.png
line_2249.png
line_225.png
line_2252.png
line_2253.png
line_2254.png
line_2255.png
line_2256.png
line_2257.png
line_2258.png
line_1978.png
line_2017.png
line_2054.png
line_2083.png
line_2108.png
line_2135.png
line_2175.png
line_2208.png
line_2230.png
line_2259.png
line_2280.png
line_2300.png
line_2329.png
line_2357.png
line_2388.png
line_2408.png
line_2446.png
line_2465.png
line_2484.png
line_2506.png
curve_1873.png
curve_2555.png
curve_525.png
line_1400.png
line_1952.png
line_2530.png
line_324.png
line_1774.png
line_1775.png
line_1777.png
line_1778.png
line_1779.png
line_178.png
line_1780.png
line_1781.png
line_1782.png
line_1784.png
line_1785.png
line_1786.png
line_1787.png
line_1788.png
line_1789.png
line_179.png
line_1790.png
line_1792.png
line_1793.png
line_1796.png
line_1797.png
line_1798.png
line_18.png
line_180.png
line_1801.png
line_827.png
line_828.png
line_829.png
line_83.png
line_830.png
line_831.png
line

curve_299.png
curve_2990.png
curve_2991.png
curve_2993.png
curve_2994.png
curve_469.png
curve_47.png
curve_470.png
curve_471.png
curve_473.png
curve_474.png
curve_475.png
curve_476.png
curve_477.png
curve_478.png
curve_479.png
curve_48.png
curve_480.png
curve_482.png
curve_483.png
curve_487.png
curve_488.png
curve_489.png
curve_49.png
curve_490.png
curve_491.png
curve_492.png
curve_493.png
curve_2094.png
curve_2095.png
curve_2097.png
curve_2098.png
curve_2099.png
curve_21.png
curve_2100.png
curve_2101.png
curve_2102.png
curve_2103.png
curve_2104.png
curve_2106.png
curve_2107.png
curve_2108.png
curve_2109.png
curve_211.png
curve_2111.png
curve_2112.png
curve_2114.png
curve_2115.png
curve_2116.png
curve_2237.png
curve_2238.png
curve_2240.png
curve_2241.png
curve_2242.png
curve_2243.png
curve_2244.png
curve_2245.png
curve_2246.png
curve_2247.png
curve_2249.png
curve_225.png
curve_2251.png
curve_2252.png
curve_2254.png
curve_2255.png
curve_2257.png
curve_2258.png
curve_226.png
curve_2261.p

curve_500.png
curve_502.png
curve_504.png
curve_505.png
curve_508.png
curve_509.png
curve_51.png
curve_510.png
curve_511.png
curve_513.png
curve_515.png
curve_516.png
curve_517.png
curve_519.png
curve_522.png
curve_524.png
curve_847.png
curve_848.png
curve_849.png
curve_851.png
curve_852.png
curve_853.png
curve_854.png
curve_855.png
curve_856.png
curve_857.png
curve_859.png
curve_86.png
curve_860.png
curve_861.png
curve_863.png
curve_864.png
curve_866.png
curve_867.png
curve_868.png
curve_869.png
curve_87.png
curve_870.png
curve_871.png
curve_872.png
curve_874.png
curve_1547.png
curve_1548.png
curve_1549.png
curve_155.png
curve_1550.png
curve_1551.png
curve_1552.png
curve_1553.png
curve_1554.png
curve_1555.png
curve_1556.png
curve_156.png
curve_1560.png
curve_1561.png
curve_1562.png
curve_1563.png
curve_1565.png
curve_1566.png
curve_1567.png
curve_1568.png
curve_157.png
curve_1570.png
line_1377.png
line_1378.png
line_138.png
line_1380.png
line_1383.png
line_1384.png
line_1385.png
line_

curve_2544.png
curve_2545.png
curve_2546.png
curve_2547.png
curve_2548.png
curve_255.png
curve_2554.png
curve_2996.png
curve_2997.png
curve_2998.png
curve_2999.png
curve_3.png
curve_30.png
curve_300.png
curve_3000.png
curve_3001.png
curve_3002.png
curve_3003.png
curve_3004.png
curve_3005.png
curve_3006.png
curve_3007.png
curve_3008.png
curve_3009.png
curve_301.png
curve_3010.png
curve_3011.png
curve_1846.png
curve_1847.png
curve_1851.png
curve_1852.png
curve_1854.png
curve_1855.png
curve_1856.png
curve_1858.png
curve_1859.png
curve_186.png
curve_1861.png
curve_1863.png
curve_1864.png
curve_1865.png
curve_1866.png
curve_1867.png
curve_1868.png
curve_1870.png
curve_1872.png
curve_1902.png
curve_1934.png
curve_1958.png
curve_1995.png
curve_2015.png
curve_204.png
curve_2064.png
curve_2093.png
curve_2117.png
curve_2143.png
curve_2162.png
curve_2200.png
curve_2236.png
curve_2264.png
curve_229.png
curve_2319.png
curve_2349.png
curve_2372.png
curve_2397.png
curve_2433.png
curve_2457.png
curve_

curve_2963.png
curve_2964.png
curve_2965.png
curve_2966.png
curve_2967.png
curve_2968.png
curve_2969.png
curve_2970.png
curve_2971.png
curve_2972.png
curve_674.png
curve_675.png
curve_677.png
curve_678.png
curve_68.png
curve_680.png
curve_681.png
curve_682.png
curve_683.png
curve_684.png
curve_687.png
curve_688.png
curve_689.png
curve_69.png
curve_690.png
curve_691.png
curve_692.png
curve_694.png
curve_695.png
curve_696.png
curve_697.png
curve_698.png
curve_699.png
curve_7.png
curve_1101.png
curve_1102.png
curve_1103.png
curve_1104.png
curve_1105.png
curve_1106.png
curve_1107.png
curve_1108.png
curve_111.png
curve_1110.png
curve_1111.png
curve_1112.png
curve_1113.png
curve_1115.png
curve_1116.png
curve_1117.png
curve_1118.png
curve_112.png
curve_1120.png
curve_1267.png
curve_1269.png
curve_1270.png
curve_1272.png
curve_1273.png
curve_1274.png
curve_1276.png
curve_1277.png
curve_1278.png
curve_1279.png
curve_128.png
curve_1281.png
curve_1282.png
curve_1283.png
curve_1284.png
curve_1287.

In [ ]:
# 所有数据 测试 
data_path = "/media/queahren/DATA/服务外包大赛数据更新/训练数据/img_train_折线"

font=cv2.FONT_HERSHEY_SIMPLEX

for k in range(1, 10):

    print(k)
    
    path = os.path.join(data_path, "%d" % k, "draw.png")
    img = cv2.imread(path)
    gray = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    
    mean, std = cv2.meanStdDev(gray)
    if mean < 50:
        gray = 255 - gray

    bw = cv2.adaptiveThreshold(
        gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 25, 25)
    img2, ctrs, hier = cv2.findContours(
        bw.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    rects = [cv2.boundingRect(ctr) for ctr in ctrs]
    
    save_path = "../data/test/" + '%d/' % k
    save_path_data = "../data/test_1/"
    if (os.path.exists(save_path) == False):
        os.mkdir(save_path)
    if (os.path.exists(save_path_data) == False):
        os.mkdir(save_path_data)    

    file0 = open(save_path_data + "%d.txt" % k,'w+') 
    new_rects = []
    i = 0
    
    for rect in rects:
        
        x, y, w, h = rect
        roi = gray[y:y + h, x:x + w]
#         print(w,h)     
        if (w > 4 and h < 20 and h > 5):
#             cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 255), 1, 1, 0);
            if w/h >= 1 and w < 22:
                res = cv2.resize(roi, (2*w, 2*h))
                res1 = resize(res[0:2*h, 0:w])
                res2 = resize(res[0:2*h, w:2*w])
                
                cv2.imwrite(save_path + '%d.jpg' % i, (res1)) 
                num, prob = predImage(save_path + '%d.jpg' % i)
                new_rects.append((x, y, round(w/2), h, num, i))
                cv2.rectangle(img, (x, y), (x+round(w/2), y+h), (0, 0, 255), 1, 1, 0);
                cv2.putText(img, '{:.0f}'.format(num), (x, y), cv2.FONT_HERSHEY_DUPLEX, h/25.0, (255, 0, 0))
                i += 1
                
                cv2.imwrite(save_path + '%d.jpg' % i, (res2))
                num, prob = predImage(save_path + '%d.jpg' % i)
                new_rects.append((x+round(w/2), y, round(w/2), h, num, i))
                cv2.rectangle(img, (x+round(w/2), y), (x+w, y+h), (0, 0, 255), 1, 1, 0);
                cv2.putText(img, '{:.0f}'.format(num), (x+round(w/2), y), cv2.FONT_HERSHEY_DUPLEX, h/25.0, (255, 0, 0))
                i += 1
                
            elif w < 15:               
                cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 255), 1, 1, 0);
                res = resize(roi)
                cv2.imwrite(save_path + '%d.jpg' % i, res)
                num, prob = predImage(save_path + '%d.jpg' % i)
                rect += tuple((num, i))
                new_rects.append(rect)    
                cv2.putText(img, '{:.0f}'.format(num), (x, y), cv2.FONT_HERSHEY_DUPLEX, h/25.0, (255, 0, 0))
                i += 1
        
    cv2.imwrite(save_path_data + "%d.jpeg" % k, img)             
    print(getY(new_rects), file=file0)
    file0.close()
    print(getX(new_rects))
    

In [ ]:
# 提取Y轴测试文件

# print(new_rects)
new_rects.sort(key=lambda tup: tup[0])  # sorts in place 原地排序
print(new_rects) 
# 提取Y轴
rects_y = []
for index in range(len(new_rects)):
    rects_y.append(new_rects[index])
#     print(new_rects[index+1][0] - new_rects[index][0])
    if (new_rects[index+1][0] - new_rects[index][0] - new_rects[index][2] > 4):
        break
print(rects_y) 


In [ ]:
# 提取Y轴测试文件

rects_y.sort(key=lambda tup: tup[1])
# print(rects_y)

rects_y_num = []
rect_y_num = []

for index in range(0, len(rects_y)-1):
    rect_y_num.append(rects_y[index])
    if (rects_y[index+1][0] - rects_y[index][0] - rects_y[index][2] > 3 or rects_y[index+1][1] - rects_y[index][1] > 2):
        rects_y_num.append(rect_y_num)
        rect_y_num = []
rect_y_num.append(rects_y[len(rects_y)-1])  
rects_y_num.append(rect_y_num)
print(rects_y_num) 


In [ ]:
# 提取Y轴测试文件
y_nums = []
for rect_y_num in rects_y_num:
    rect_y_num.sort(key=lambda tup: tup[0], reverse=True)
    y_num = 0
    y_pic = 0
    for i in range(0, len(rect_y_num)):
        y_num += rect_y_num[i][4] * pow(10, i)
        y_pic += (2*rect_y_num[i][1] + rect_y_num[i][3])/2
    y_nums.append((y_num, y_pic/len(rect_y_num)))          
print(y_nums)

In [ ]:
# 提取X轴测试文件

# print(new_rects)
new_rects.sort(key=lambda tup: tup[1], reverse=True)  # sorts in place 原地排序
print(new_rects) 
# 提取X轴
rects_x = []
for index in range(len(new_rects)):
    rects_x.append(new_rects[index])
#     print(new_rects[index+1][0] - new_rects[index][0])
    if abs(new_rects[index+1][1] - new_rects[index][1]) > 4:
        break
print(rects_x) 

In [ ]:
# 提取X轴测试文件

rects_x.sort(key=lambda tup: tup[0])
print(rects_x)

rects_x_num = []
rect_x_num = []

for index in range(0, len(rects_x)-1):
    rect_x_num.append(rects_x[index])
    if (rects_x[index+1][0] - rects_x[index][0] - rects_x[index][2] > 3 or rects_x[index+1][1] - rects_x[index][1] > 2):
        rects_x_num.append(rect_x_num)
        rect_x_num = []
rect_x_num.append(rects_x[len(rects_x)-1])  
rects_x_num.append(rect_x_num)
print(rects_x_num) 

In [ ]:
# 提取X轴测试文件
x_nums = []
for rect_x_num in rects_x_num:
    rect_x_num.sort(key=lambda tup: tup[0], reverse=True)
    x_num = 0
    x_pic = 0
    for i in range(0, len(rect_x_num)):
        x_num += rect_x_num[i][4] * pow(10, i)
        x_pic += (2*rect_x_num[i][0] + rect_x_num[i][2])/2
    x_nums.append((x_num, x_pic/len(rect_x_num)))          
print(x_nums)